In [1]:
# Diagnostics

import os
print(f"The Current Working Directory is: {os.getcwd()}")

The Current Working Directory is: C:\Users\kincaidn\OneDrive - City of Austin\Documents\GitHub\chiron-guild-core\notebooks


In [ ]:
# Cell 1: Setup & Initialization (Local Environment Version)

import os
from dotenv import load_dotenv
import google.generativeai as genai

# --- Configuration ---
# Load environment variables from the .env file in the root directory
load_dotenv()

# Get the API key from the environment
api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    print("🔴 ERROR: GEMINI_API_KEY not found in .env file.")
    print("Please create a .env file in the repository root with GEMINI_API_KEY='your-key'.")
else:
    genai.configure(api_key=api_key)

# --- Guild Protocol Functions ---

def find_project_root(marker_file='.gitignore'):
    """Finds the project's root directory by searching upwards for a marker file."""
    current_path = os.getcwd()
    while current_path != os.path.dirname(current_path): # Stop at the filesystem root
        if marker_file in os.listdir(current_path):
            return current_path
        current_path = os.path.dirname(current_path)
    raise FileNotFoundError(f"Project root marker '{marker_file}' not found.")

def load_protocol_file(root_path, relative_file_path):
    """Loads a specific protocol document using an absolute path from the project root."""
    full_path = os.path.join(root_path, relative_file_path)
    try:
        with open(full_path, 'r', encoding='utf-8') as f:
            print(f"✅  Context file loaded: {relative_file_path}")
            return f.read()
    except FileNotFoundError:
        print(f"🔴 ERROR: Protocol file not found at '{full_path}'.")
        return None

def save_chat_history(chat_session, file_path):
    """Saves the chat history to a JSON file."""
    history = [
        {'role': msg.role, 'parts': [part.text for part in msg.parts]}
        for msg in chat_session.history
    ]
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(history, f, indent=2)
        print(f"✅ Chat history saved to {file_path}")
    except Exception as e:
        print(f"🔴 ERROR saving history: {e}")

def load_chat_history(root_path, relative_file_path):
    """Loads and sanitizes a chat history from a JSON file."""
    full_path = os.path.join(root_path, relative_file_path)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            history = json.load(f)
    except FileNotFoundError:
        print(f"🟡 INFO: No previous history found at {file_path}. Starting a new session.")
        return None
    except Exception as e:
        print(f"🔴 ERROR loading or parsing history file: {e}")
        return None

    # --- THIS IS THE CRITICAL SANITIZATION STEP ---
    valid_roles = ['user', 'model']
    sanitized_history = []
    
    for message in history:
        if isinstance(message, dict) and 'role' in message and message['role'] in ['user', 'model']:
            sanitized_history.append(message)
    return sanitized_history

    # Ensure the history alternates correctly between user and model
    if sanitized_history:
        # The first message must be from a 'user'
        if sanitized_history[0]['role'] != 'user':
            print("🔴 ERROR: Sanitized history does not start with a 'user' role. Starting a new session.")
            return None

    print(f"✅ History sanitized. Loaded {len(sanitized_history)} valid messages.")
    return sanitized_history

# The full, corrected function in Cell 1

def initialize_guild_chat(system_instruction, history=None):
    """Initializes a new chat session with system instruction and optional history."""
    
    # Define the generation_config for more detailed responses
    generation_config = {
      "temperature": 0.7,
      "top_p": 1,
      "top_k": 32,
      "max_output_tokens": 8192,
    }

    # This is the corrected line
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash-preview-05-20",
        system_instruction=system_instruction,
        generation_config=generation_config
    )
    
    # Pass the loaded history to the start_chat method
    chat = model.start_chat(history=history if history else None)
    return chat
    
    # Pass the loaded history to the start_chat method
    chat = model.start_chat(history=history if history else None)
    return chat
    
    generation_config = {
        "temperature": 0.7,
        "top_p": 1,
        "top_k": 32,
        "max_output_tokens": 8192, # Increase the max token output significantly
}
    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash-preview-05-20", # Corrected Model Name
        system_instruction=system_instruction,
        generation_config=generation_config
    )
    chat = model.start_chat()
    return chat

# --- Main Initialization Step ---
print("🛡️  Initializing Guild Interface AI...")

try:
    # Find the project root directory dynamically
    PROJECT_ROOT = find_project_root()
    print(f"✅  Project root found at: {PROJECT_ROOT}")

    # Define paths relative to the project root
    ORACLE_PROMPT_PATH = os.path.join("_Admin & Core Docs", "Protocols-Frameworks", "guild_oracle_prompt.md")
    COPILOT_CONTEXT_PATH = os.path.join("_Admin & Core Docs", "Protocols-Frameworks", "Copilot_Context_Protocol.md")
    CHAT_LOG_PATH = os.path.join("chat_logs", "portfolio_project_chat.json")
    loaded_history = load_chat_history(PROJECT_ROOT, CHAT_LOG_PATH)

    # Load all necessary components using the root path
    oracle_persona_prompt = load_protocol_file(PROJECT_ROOT, ORACLE_PROMPT_PATH)
    copilot_foundational_context = load_protocol_file(PROJECT_ROOT, COPILOT_CONTEXT_PATH)

    # Combine the prompts into a single, comprehensive system instruction
    if oracle_persona_prompt and copilot_foundational_context:
        full_system_instruction = f"""
{oracle_persona_prompt}

---
## Supplemental Foundational Context
Below is the foundational context for the Chiron Guild, which you must use to inform all your responses.
{copilot_foundational_context}
"""
        
        # Pass the combined instruction and loaded history to the session
        guild_chat_session = initialize_guild_chat(full_system_instruction, history=loaded_history)
        
        if guild_chat_session:
            print("✅  Initialization Complete. Oracle is calibrated and online.")
            print("➡️  Proceed to run the UI cells to begin the chat session.")
    else:
        print("🔴  Initialization Failed. One or more core protocol files could not be loaded.")

except FileNotFoundError as e:
    print(f"🔴 CRITICAL ERROR: {e}")

🛡️  Initializing Guild Interface AI...
✅  Project root found at: C:\Users\kincaidn\OneDrive - City of Austin\Documents\GitHub\chiron-guild-core
🔴 ERROR: Protocol file not found at 'C:\Users\kincaidn\OneDrive - City of Austin\Documents\GitHub\chiron-guild-core\_Admin & Core Docs\Protocols-Frameworks\prompts\guild_oracle_prompt.md'.
✅  Context file loaded: _Admin & Core Docs\Protocols-Frameworks\Copilot_Context_Protocol.md
🔴 ERROR loading or parsing history file: name 'file_path' is not defined
🔴  Initialization Failed. One or more core protocol files could not be loaded.


In [ ]:
# Cell 2: UI Setup
import ipywidgets as widgets
from IPython.display import display, HTML

# Create the UI components
chat_history_view = widgets.Output(layout={'border': '1px solid black', 'height': '400px', 'overflow_y': 'auto'})
prompt_input = widgets.Text(placeholder='Enter your message...')
send_button = widgets.Button(description='Send')
loading_spinner = widgets.HTML("<p><i>⚫️ [Guild AI]: Compiling response...</i></p>", layout={'display': 'none'})

def display_message(who, message, is_html=False):
    """Appends a message to the chat history view."""
    
    if is_html:
        # If the message is already HTML, just add the sender info and display directly.
        # This prevents wrapping the AI's markdown-converted HTML in another <p> tag.
        color = "green"
        icon = "🟢"
        formatted_message = f'<div style="margin-left: 5px;"><p style="color:{color}; margin-bottom: 0px;"><strong>{icon} [Guild AI]:</strong></p>{message}</div>'
    else:
        # For plain text messages (like the user's), wrap them in a <p> tag as before.
        color = "royalblue"
        icon = "🔵"
        formatted_message = f'<p style="color:{color}; margin-left: 5px;"><strong>{icon} [{who}]:</strong> {message}</p>'
    
    # The IPython.display.HTML object is used to render the string as HTML
    chat_history_view.append_display_data(HTML(formatted_message))

display(HTML("""
<style>
    .widget-text textarea { width: 100% !important; }
    .widget-button button { background-color: #4CAF50 !important; color: white !important; }
</style>
"""))

print("✅  Chat Interface Initialized. Enter your prompt below.")
display(chat_history_view, widgets.HBox([prompt_input, send_button]), loading_spinner)

In [16]:
# Cell 3: Context Toolkit

CONTEXT_FILES = {
    "manifesto": "_Admin & Core Docs/Protocols-Frameworks/GUILD_MANIFESTO.md",
    "taxonomy": "_Admin & Core Docs/Protocols-Frameworks/taxonomy_framework.md",
    "decomposition": "project_decomposition.md",
    "op_protocols": "_Admin & Core Docs/Protocols-Frameworks/GUILD_OP_PROTOCOLS.md",
    "registry": "_Admin & Core Docs/registry/operative_registry.json"
}

def get_context(keyword):
    """Retrieves the content of a specific context file."""
    file_path = CONTEXT_FILES.get(keyword.lower())
    if not file_path:
        return f"ERROR: Unknown context keyword '{keyword}'. Valid keywords are: {list(CONTEXT_FILES.keys())}"
    
    try:
        # Adjust pathing as needed
        full_path = os.path.join(os.getcwd(), '..', file_path)
        if not os.path.exists(full_path):
             full_path = os.path.join(os.getcwd(), file_path)
        
        with open(full_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        return f"ERROR: Could not load file for '{keyword}': {e}"

In [17]:
# Cell 4: UI Logic (The "Backend" for the UI)

def on_send_button_clicked(b):
    prompt = prompt_input.value
    if not prompt.strip():
        return # Do nothing if input is empty
    
    # Display the user's message and show loading spinner
    display_message("Operative Kin-Caid", prompt)
    prompt_input.value = "" # Clear the input box
    loading_spinner.layout.display = 'block'

    final_prompt = prompt
    
    if prompt.strip().lower().startswith('!context'):
        parts = prompt.strip().split()
        if len(parts) > 1:
            keyword = parts[1]
            
            # Extract the actual user question, which is everything AFTER the command and keyword
            actual_question = " ".join(parts[2:])
            if not actual_question:
                actual_question = f"Please summarize the key points of the '{keyword}' document." # Default question

            context_content = get_context(keyword)
            display_message("SYSTEM", f"<i>Injecting context from '{keyword}'...</i>")

            # Create a new, more forceful prompt structure
            final_prompt = f"""
An Operative has asked the following question:
"{actual_question}"

To formulate your answer, you MUST use the following reference document that has been loaded into your context for this specific turn. Do not claim you cannot access it; it is provided below.

<CONTEXT_DOCUMENT>
---
FILE: {keyword}
---
{context_content}
</CONTEXT_DOCUMENT>

Based on the content of the provided document, answer the Operative's question.
"""
    
    try:
        # Send message to the AI
        response = guild_chat_session.send_message(final_prompt)
        
        # Display AI's response (using HTML to render markdown)
        from markdown import markdown
        html_response = markdown(response.text)
        display_message("Guild AI", html_response, is_html=True)

    except Exception as e:
        display_message("SYSTEM", f"<strong>🔴 ERROR:</strong> {e}")
    finally:
        # Hide loading spinner
        loading_spinner.layout.display = 'none'

send_button.on_click(on_send_button_clicked)

Use these commands at the start of a chat to add context docs:

!context manifesto
!context taxonomy
!context decomposition
!context op_protocols
!context registry

Use this to generate chat history

In [5]:
save_chat_history(guild_chat_session, 'chat_logs/portfolio_project_chat.json')

🔴 ERROR saving history: [Errno 2] No such file or directory: 'chat_logs/portfolio_project_chat.json'
